# O que é COVID-19?
Os coronavírus são uma grande família de vírus que podem causar doenças respiratórias em humanos, desde resfriados comuns a condições mais graves, como Síndrome Respiratória Aguda Grave (SARS) e Síndrome Respiratória do Oriente Médio (MERS) .1 'Novo coronavírus' é um novo, anteriormente cepa não identificada de coronavírus. O novo coronavírus envolvido no surto atual foi denominado SARS-CoV-2 pela Organização Mundial da Saúde (OMS). 3A doença que causa foi denominada “doença coronavírus 2019” (ou “COVID-19”). `

Com o mundo parando, milhares perdendo suas vidas e economias em queda livre, o surto do novo coronavírus COVID-19 SARS-CoV-2 roubou todo o planeta agora.
Durante A PANDEMIA, decidi examinar alguns repositórios de dados e comecei a organizar e plotar os dados. Depois de um tempo, decidi criar um painel. 

# Reúna dados de fontes confiáveis ​​e atualizadas
Eu reuni meus dados do repositório de dados do Novel Coronavirus COVID-19 de 2019 por John Hopkins CSSE.

Os dados aqui são atualizados a cada 24 horas a partir de fontes oficiais. Eu acessei os arquivos brutos do csse_covid_19_time_seriesdiretório.
Aqui está como estou lendo o arquivo usando o read_csv(),  método do pandas :

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

In [2]:
# carregando dados direto da fonte:
morte_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmado_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recuperado_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
pais_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [3]:
 confirmado_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,7/17/21,7/18/21,7/19/21,7/20/21,7/21/21,7/22/21,7/23/21,7/24/21,7/25/21,7/26/21,7/27/21,7/28/21,7/29/21,7/30/21,7/31/21,8/1/21,8/2/21,8/3/21,8/4/21,8/5/21,8/6/21,8/7/21,8/8/21,8/9/21,8/10/21,8/11/21,8/12/21,8/13/21,8/14/21,8/15/21,8/16/21,8/17/21,8/18/21,8/19/21,8/20/21,8/21/21,8/22/21,8/23/21,8/24/21,8/25/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,137853,137853,141489,142414,142414,143183,143439,143439,143439,144285,145008,145552,145996,146523,146523,146523,147985,148572,148933,149361,149810,149810,149810,150778,151013,151291,151563,151770,151770,151770,152142,152243,152363,152411,152448,152448,152448,152583,152660,152722
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,132665,132686,132697,132740,132763,132797,132828,132853,132875,132891,132922,132952,132999,133036,133081,133121,133146,133211,133310,133442,133591,133730,133912,133981,134201,134487,134761,135140,135550,135947,136147,136598,137075,137597,138132,138790,139324,139721,140521,141365
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,152210,153309,154486,155784,157005,158213,159563,160868,162155,163660,165204,167131,168668,170189,171392,172564,173922,175229,176724,178013,179216,180356,181376,182368,183347,184191,185042,185902,186655,187258,187968,188663,189384,190078,190656,191171,191583,192089,192626,193171
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,14273,14273,14359,14379,14379,14464,14498,14498,14498,14577,14586,14586,14655,14678,14678,14678,14747,14766,14797,14809,14836,14836,14836,14836,14873,14891,14908,14924,14924,14924,14954,14960,14976,14981,14988,14988,14988,15002,15003,15014
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,40707,40805,40906,41061,41227,41405,41629,41736,41780,41879,42110,42288,42486,42646,42777,42815,42970,43070,43158,43269,43487,43592,43662,43747,43890,43998,44174,44328,44534,44617,44739,44972,45175,45325,45583,45817,45945,46076,46340,46539


In [4]:
recuperado_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,7/17/21,7/18/21,7/19/21,7/20/21,7/21/21,7/22/21,7/23/21,7/24/21,7/25/21,7/26/21,7/27/21,7/28/21,7/29/21,7/30/21,7/31/21,8/1/21,8/2/21,8/3/21,8/4/21,8/5/21,8/6/21,8/7/21,8/8/21,8/9/21,8/10/21,8/11/21,8/12/21,8/13/21,8/14/21,8/15/21,8/16/21,8/17/21,8/18/21,8/19/21,8/20/21,8/21/21,8/22/21,8/23/21,8/24/21,8/25/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,130072,130074,130081,130086,130097,130109,130118,130125,130139,130152,130166,130174,130187,130205,130222,130243,130256,130291,130314,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,104992,105604,106337,107041,107776,108537,109349,109951,110577,111322,112050,112900,113707,114531,115276,116009,116770,117557,118409,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,13844,13844,13897,13930,13930,13988,13988,13988,13988,14077,14113,14113,14180,14210,14210,14210,14296,14348,14380,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,34732,34755,34790,34857,34893,35082,35284,35423,35474,35686,35742,35964,36025,36708,37255,37397,38360,39389,39582,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
morte_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,7/17/21,7/18/21,7/19/21,7/20/21,7/21/21,7/22/21,7/23/21,7/24/21,7/25/21,7/26/21,7/27/21,7/28/21,7/29/21,7/30/21,7/31/21,8/1/21,8/2/21,8/3/21,8/4/21,8/5/21,8/6/21,8/7/21,8/8/21,8/9/21,8/10/21,8/11/21,8/12/21,8/13/21,8/14/21,8/15/21,8/16/21,8/17/21,8/18/21,8/19/21,8/20/21,8/21/21,8/22/21,8/23/21,8/24/21,8/25/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,5983,5983,6213,6266,6266,6325,6357,6357,6357,6477,6515,6577,6615,6677,6677,6677,6774,6804,6836,6864,6879,6879,6879,6936,6961,6978,6988,7000,7000,7000,7025,7031,7043,7047,7054,7054,7054,7076,7083,7090
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2456,2456,2456,2456,2456,2456,2456,2456,2456,2456,2456,2457,2457,2457,2457,2457,2457,2457,2457,2457,2458,2459,2459,2460,2460,2460,2460,2461,2463,2464,2466,2468,2471,2473,2475,2477,2478,2478,2480,2483
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3922,3938,3956,3979,3994,4008,4026,4042,4063,4087,4112,4161,4189,4219,4254,4291,4329,4370,4404,4441,4487,4520,4550,4578,4618,4654,4695,4730,4766,4794,4830,4867,4898,4929,4957,4984,5004,5034,5063,5096
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,127,127,127,127,127,127,127,127,127,127,127,127,127,128,128,128,128,128,128,128,128,128,128,128,129,129,129,129,129,129,129,129,129,129,129,129,129,130,130,130
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,954,957,969,970,973,977,981,982,984,991,994,1000,1003,1008,1011,1016,1018,1022,1026,1032,1042,1044,1049,1053,1057,1063,1070,1082,1086,1100,1103,1118,1125,1133,1138,1140,1153,1163,1166,1176


In [6]:
pais_df.head()

,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Cases_28_Days,Deaths_28_Days
0,Afghanistan,2021-08-26 11:21:46,33.93911,67.709953,152722,7090,NaN,NaN,392.315717,NaN,NaN,4.642422,4,AFG,7170,513
1,Albania,2021-08-26 11:21:46,41.15330,20.168300,141365,2483,NaN,NaN,4912.259365,NaN,NaN,1.756446,8,ALB,8413,26
2,Algeria,2021-08-26 11:21:46,28.03390,1.659600,193171,5096,NaN,NaN,440.516318,NaN,NaN,2.638077,12,DZA,26040,935
3,Andorra,2021-08-26 11:21:46,42.50630,1.521800,15014,130,NaN,NaN,19431.825535,NaN,NaN,0.865859,20,AND,428,3
4,Angola,2021-08-26 11:21:46,-11.20270,17.873900,46539,1176,NaN,NaN,141.601109,NaN,NaN,2.526913,24,AGO,4251,176


In [7]:
# LIMPEZA DE DADOS

# renomeando os nomes das colunas df para minúsculas
pais_df.columns = map(str.lower, pais_df.columns)
confirmado_df.columns = map(str.lower, confirmado_df.columns)
morte_df.columns = map(str.lower, morte_df.columns)
recuperado_df.columns = map(str.lower, recuperado_df.columns)

# alterar província / estado para estado e país / região para país
confirmado_df = confirmado_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recuperado_df = recuperado_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
morte_df = morte_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
pais_df = pais_df.rename(columns={'country_region': 'country'})
pais_df.head()

,country,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3,cases_28_days,deaths_28_days
0,Afghanistan,2021-08-26 11:21:46,33.93911,67.709953,152722,7090,NaN,NaN,392.315717,NaN,NaN,4.642422,4,AFG,7170,513
1,Albania,2021-08-26 11:21:46,41.15330,20.168300,141365,2483,NaN,NaN,4912.259365,NaN,NaN,1.756446,8,ALB,8413,26
2,Algeria,2021-08-26 11:21:46,28.03390,1.659600,193171,5096,NaN,NaN,440.516318,NaN,NaN,2.638077,12,DZA,26040,935
3,Andorra,2021-08-26 11:21:46,42.50630,1.521800,15014,130,NaN,NaN,19431.825535,NaN,NaN,0.865859,20,AND,428,3
4,Angola,2021-08-26 11:21:46,-11.20270,17.873900,46539,1176,NaN,NaN,141.601109,NaN,NaN,2.526913,24,AGO,4251,176


In [8]:
# número total de casos confirmados, óbitos e recuperados
confirmado_total = int(pais_df['confirmed'].sum())
morte_total = int(pais_df['deaths'].sum())
recuperado_total = int(pais_df['recovered'].sum())
ativos_total = int(pais_df['active'].sum())

In [9]:
# exibindo as estatísticas totais

display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: #fff; font-size:30px;'> Confirmados: "  + str(confirmado_total) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Óbitos: " + str(morte_total) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recuperados: " + str(recuperado_total) + "</span>"+
             "</div>")
       )

# Casos confirmados / óbito / recuperados COVID-19 por países

# Insira o número de países para os quais deseja os dados



In [10]:
# classificando os valores por ordem decrescente confirmada
# pais_df.sort_values('confirmed', ascending= False).head(10).style.background_gradient(cmap='copper')
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    r = 'background-color: red'
    y = 'background-color: purple'
    g = 'background-color: grey'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = y
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    
    return df1

def show_latest_cases(n):
    n = int(n)
    return pais_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)

interact(show_latest_cases, n='10')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # descomente, execute a célula novamente - então o gráfico / figura desaparece
widgets.VBox([fig], layout=ipywLayout)


interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [11]:
classificado_pais_df = pais_df.sort_values('confirmed', ascending= False)

# verificar os países mais afetados

plote os dados que temos usando a biblioteca Plotly . Traçamos os mesmos países mais atingidos junto com uma opção de controle deslizante para alterar o número de países que queremos exibir:

O Plotly renderiza gráficos interativos de alta qualidade. Veja como os dados se parecem agora:

In [12]:
# # traçando os 20 países mais atingidos

def bubble_chart(n):
    fig = px.scatter(classificado_pais_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Piores países atingidos",
    xaxis_title="Paises",
    yaxis_title="Casos Confirmados",
    width = 700
    )
    fig.show();

interact(bubble_chart, n=10)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [13]:
def plot_cases_of_a_country(country):
    labels = ['confirmados', 'obitos']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4, 5]
    
    df_list = [confirmado_df, morte_df]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)
        print(i)
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
    
    fig.update_layout(
        title="COVID 19 casos " + country,
        xaxis_title='Data',
        yaxis_title='Nº de casos confirmados',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();

# Verifique os detalhes do seu país ou do mundo
Insira o nome do seu país (em letras maiúsculas (por exemplo, Itália)) e mundo para o total de casos

In [14]:
interact(plot_cases_of_a_country, country='World')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' 
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Text(value='World', description='country'), Output()), _dom_classes=('widget-interact',)…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

# 10 países mais atingidos - Casos confirmados

In [15]:
px.bar(
    classificado_pais_df.head(10),
    x = "country",
    y = "confirmed",
    title= "Top 10 paises mais afetados",
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

# 10 países mais atingidos - Casos de morte

In [16]:
px.bar(
    classificado_pais_df.head(10),
    x = "country",
    y = "deaths",
    title= "Top 10 paises com mais óbitos", 
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

# Países mais atingidos - casos de recuperação

In [17]:
px.bar(
    classificado_pais_df.head(10),
    x = "country",
    y = "recovered",
    title= "Top 10 paises com mais recuperação", 
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)